# Finished Components

Write the code on personal jupyterhub, and test on there, when done, paste your function into a new cell on here, and create a cell underneath to show a test of your function that works.

In [34]:
# PUT ALL IMPORTS FOR YOUR FUNCTIONS HERE
import numpy as np
import scipy.stats as stats
import os
import pwd
import re
import argparse

USER = pwd.getpwuid( os.getuid() )[ 0 ]

## Argument Parser

In [ ]:
# Define the tags, control tags, and output as command-line arguments
parser = argparse.ArgumentParser()
parser.add_argument('tags_tsv', type=str, help='input tags tsv file path')
parser.add_argument('control_tags', type=str, help='input control tags tsv file path')
parser.add_argument('-O', '--output', type=str, help='output file path')
args = parser.parse_args()

#We can call args.tags_tsv, args.control_tags, or args.output

### processTagsTSV
- processes tags from inputted TSV file into a dictionary of dictionaries, with the first key being chromosome pointing to a dictionary with keys bp position and value number of reads at that position

In [35]:
# Global variable allCounts (dictionary of dictionaries), so that you do not have to keep running the cell
# after initial run to test code that relies on it.
def processTagTSVs(dir_name):
    allCounts = {}
    files = os.listdir(dir_name)
    for file_name in files:
        if file_name.endswith(".tags.tsv"):
            fileCounts = {}
            with open(os.path.join(dir_name, file_name)) as tsv:
                for line in tsv:
                    line = line.strip("\n")
                    line = line.split("\t")
                    for i in range(int(line[2]), int(line[2]) + int(line[-1])):
                        key = i
                        if key not in fileCounts.keys():
                            fileCounts[key] = int(float(line[4]))
                        else:
                            fileCounts.update({key : fileCounts[key]+int(float(line[4]))})
            fileCounts = {key : value for key, value in fileCounts.items() if value > 0}
            sortKeys = list(fileCounts.keys())
            sortKeys.sort()
            fileCounts = {i : fileCounts[i] for i in sortKeys}
            allCounts[file_name.split(".")[0]] = fileCounts
    
    sortKeys = list(allCounts.keys())
    sortKeys.sort()
    allCounts = {i : allCounts[i] for i in sortKeys}

    return allCounts

In [36]:
#global allCounts
def testProcessTagsTSVs():

    allCounts = processTagTSVs("/home/%s/teams/25/tagdirs/Sox2"%USER)
    i = 0
    print(len(allCounts))
    for chrom, cr_dict in allCounts.items():
        i = 0
        for x, y in cr_dict.items():
            print(x,y)
            i+=1
            if i > 100:
                break
                
#testProcessTagsTSVs()

### PLACEHOLDER FUNCTION (REPLACE WITH WHATEVER IS NEXT)

In [4]:
def placeholder():
    return

In [5]:
def testPlaceholder():
    return

## BED File output:

Will take in the input of a list of start, end positions for each peak. This will output a BED file in the format that each line is the start position, tab, end position, tab, p-value, tab.

In [6]:
paired_list = [(3,78),(99,174),(201,276)]

def makeBED(paired_list):
    file = open("myfile.bed", "w")
    with file as f:
            for pair in paired_list:
                f.write(str(pair[0]) + "\t" + str(pair[1]) + "\t" + str(1) + "\n")

makeBED(paired_list)

## Calculate Fold Change:

In [37]:

def calcMaxFoldChange(windowStart, windowEnd, cr_dict, input_cr_dict):
    #maxVal = 0
    TFcounter = 0
    ControlCounter = 0
    #in between startPos and startPos+windowSize, find the position with MAX value
    for i in range(windowStart, windowEnd):
        #check if it is the new max
        if i in cr_dict.keys():
            TFcounter += cr_dict[i]
        if i in input_cr_dict.keys():
            ControlCounter += input_cr_dict[i]
    print ("TF: "+ str(TFcounter))
    print ("CC: " + str(ControlCounter))
    return TFcounter/ControlCounter

In [38]:
def findPeaks(cr_dict,input_cr_dict, fcThreshold, windowSize):
    possiblePeaks = []
    count = 0
    swindowStart = 0
    for key, value in cr_dict.items():
        if count == 0:
            windowStart = value 
            break
    """
        print(str(key) + "\t" + str(value))
        count+=1
        if count == 100:
            break
    for key, value in input_cr_dict.items():
        print(str(key) + "\t" + str(value))
        count+=1
        if count == 100:
            break"""
    
    windowEnd = windowStart + windowSize
    print (windowStart)
    for key, value in input_cr_dict.items():  
        print ("key: " + str(key) + " value: " + str(value))
    #hunter code outline below:
    #do it per chromosome
    #given a window size, that is how much you check. incrementn i by window size
    #temporarily work only on chr17
    #start window is first position

    for pos in cr_dict:

        
        print ("start: " + str(windowStart))
        print ("end: " + str(windowEnd))
 
        #print (pos)
        #run maxFoldChange

        maxFoldChange = calcMaxFoldChange(windowStart, windowEnd, cr_dict, input_cr_dict)
    
        #check if this maxFoldChange is above our threshold
        if (maxFoldChange >= fcThreshold):
            possiblePeaks.append((windowStart,windowEnd))
            print (maxFoldChange)
        #if position is still in the window, since we already ran it, continue
        if pos < windowEnd:
            continue
        else:
            #update window otherwise
            windowStart = pos
            windowEnd = pos + windowSize
    
    return possiblePeaks
       

In [39]:
#process files for testing
#RUN THIS ONLY ONCE
def processFiles():
    Sox2Counts = processTagTSVs("/home/%s/teams/25/tagdirs/Sox2"%USER)
    controlCounts = processTagTSVs("/home/%s/teams/25/tagdirs/input"%USER)
    return Sox2Counts, controlCounts
Sox2Counts, controlCounts = processFiles()

In [40]:
def testPeaks():
    cr_dict_sox2 = {}
    cr_dict_input = {}
    #get sox2 stuff for testing
    for chrom, cr_dict in Sox2Counts.items():
        if chrom == "2":
            cr_dict_sox2 = cr_dict
            break
    #get input stuff for testing
    for chrom, cr_dict in controlCounts.items():
        if chrom == "2":
            cr_dict_input = cr_dict
            break
            
    possiblePeaks = findPeaks(cr_dict_sox2, cr_dict_input, fcThreshold = 2, windowSize = 10)
    print (possiblePeaks)
testPeaks()

1
key: 3051609 value: 1
key: 3051610 value: 1
key: 3051611 value: 1
key: 3051612 value: 1
key: 3051613 value: 1
key: 3051614 value: 1
key: 3051615 value: 1
key: 3051616 value: 1
key: 3051617 value: 1
key: 3051618 value: 1
key: 3051619 value: 1
key: 3051620 value: 1
key: 3051621 value: 1
key: 3051622 value: 1
key: 3051623 value: 1
key: 3051624 value: 1
key: 3051625 value: 1
key: 3051626 value: 1
key: 3051627 value: 2
key: 3051628 value: 2
key: 3051629 value: 2
key: 3051630 value: 2
key: 3051631 value: 2
key: 3051632 value: 2
key: 3051633 value: 2
key: 3051634 value: 2
key: 3051635 value: 2
key: 3051636 value: 2
key: 3051637 value: 2
key: 3051638 value: 2
key: 3051639 value: 2
key: 3051640 value: 2
key: 3051641 value: 2
key: 3051642 value: 2
key: 3051643 value: 2
key: 3051644 value: 2
key: 3051645 value: 2
key: 3051646 value: 2
key: 3051647 value: 2
key: 3051648 value: 2
key: 3051649 value: 2
key: 3051650 value: 2
key: 3051651 value: 2
key: 3051652 value: 2
key: 3051653 value: 2
key: 305

ZeroDivisionError: division by zero

In [ ]:
#ONLY RUN THIS ONCE WHEN OPENING THE NOTEBOOK
def testProcessTagsTSVsInput():
    # CHANGE USER BEFORE RUNNING OR IT WONT WORK FOR YOU
    global inputCounts
    #chr: cr_dict
    inputCounts = processTagTSVs("/home/hbcheng/teams/25/tagdirs/input")
    i = 0
    print(len(inputCounts))
    for chrom, cr_dict in inputCounts.items():
        i = 0
        for x, y in cr_dict.items():
            print(x,y)
            i+=1
            if i > 100:
                break
testProcessTagsTSVsInput()